In [1]:
from ortools.linear_solver import pywraplp

In [2]:
def load_data(path = "TSP.exe"):
    with open(path, 'r') as f:
        inputData = f.readlines()
    
    N = int(inputData[0].strip())
    node_list=[]
    for node in inputData[1:]:
        #del '\n' 
        node = node.strip()
        #split by ' '
        node = node.split(' ')
        node_list.append((int(node[0]), int(node[1])))

    return node_list, N

node_list, N = load_data()

In [3]:
import numpy as np

def DistanceMatrix(cities, n):
    dis_matrix = np.zeros([n,n])
    
    for i in range(n):
        for j in range(i+1, n):
            a = np.array(cities[i])
            b = np.array(cities[j])
            c = a - b

            dis_matrix[i, j] = np.sqrt(np.sum(c*c))
            dis_matrix[j, i] = dis_matrix[i, j]
    
    return np.around(dis_matrix, 2)

c = DistanceMatrix(node_list, N)

In [21]:
# Miller-Tucker-Zemlin
solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()

X = []
Y = []

for i in range(N):
    X_i = []
    y_i = solver.IntVar(0, N, 'y({})'.format(i))
    Y.append(y_i)
    for j in range(N):
        x = solver.IntVar(0, 1, 'x({},{})'.format(i, j))
        X_i.append(x)
    X.append(X_i)

for i in range(N):
    s_i_j = 0
    s_j_i = 0
    for j in range(N):
        if i != j:
            s_i_j += X[i][j]
            s_j_i += X[j][i]

            if i >= 1:
                solver.Add(Y[j] - Y[i] + N*(1 - X[i][j]) >= 1)


    solver.Add(s_i_j == 1)
    solver.Add(s_j_i == 1)

objective = solver.Objective()
for i in range(N):
    for j in range(N):
        objective.SetCoefficient(X[i][j], c[i][j])

objective.SetMinimization()


status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())

else:
    print('The problem does not have an optimal solution.')

print('\nNumber of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())


Solution:
Objective value = 61.86

Number of variables = 90
Number of constraints = 82

Advanced usage:
Problem solved in 277.000000 milliseconds
Problem solved in 1190 iterations
Problem solved in 5 branch-and-bound nodes


In [26]:
# Add SEC

solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()

X = []

for i in range(N):
    X_i = []
    for j in range(N):
        x = solver.IntVar(0, 1, 'x({},{})'.format(i, j))
        X_i.append(x)
    X.append(X_i)

for i in range(N):
    s_i_j = 0
    s_j_i = 0
    for j in range(N):
        if i != j:
            s_i_j += X[i][j]
            s_j_i += X[j][i]

    solver.Add(s_i_j == 1)
    solver.Add(s_j_i == 1)

objective = solver.Objective()
for i in range(N):
    for j in range(N):
        objective.SetCoefficient(X[i][j], c[i][j])

objective.SetMinimization()


while(True):
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        #check
        mark = [0 for i in range(N)]
        curr = 0
        mark[0] = 1
        S = [0]
        for k in range(N-1):
            for i in range(N):
                if X[curr][i].solution_value() == 1:
                    curr = i
                    break

            if mark[curr] == 0:
                mark[curr] = 1
                S.append(curr)
            else:
                break

        if len(S) == N:
            print('Solution:')
            print('Objective value =', solver.Objective().Value())
            break

        total = 0
        for i in S:
            for j in S:
                if i != j:
                    total += X[i][j]

        solver.Add(total <= len(S) - 1)
                    
    else:
        print('The problem does not have an optimal solution.')
        break

print('\nNumber of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())


Solution:
Objective value = 61.86

Number of variables = 81
Number of constraints = 27

Advanced usage:
Problem solved in 79.000000 milliseconds
